In [1]:
!pip install opendatasets

import opendatasets as od

od.download("https://www.kaggle.com/datasets/mahdimashayekhi/fake-news-detection-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: 
Your Kaggle username: 44eca3b3b4214961f2495abdfbd8ba4e
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/mahdimashayekhi/fake-news-detection-dataset


100%|██████████| 11.2M/11.2M [00:00<00:00, 1.09GB/s]

In [40]:
# Some important Lib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier

In [41]:
# First look of our dataset
df = pd.read_csv("/content/fake-news-detection-dataset/fake_news_dataset.csv")

In [42]:
df.head()

,title,text,date,source,author,category,label
0,Foreign Democrat final.,more tax development both store agreement lawy...,2023-03-10,NY Times,Paula George,Politics,real
1,To offer down resource great point.,probably guess western behind likely next inve...,2022-05-25,Fox News,Joseph Hill,Politics,fake
2,Himself church myself carry.,them identify forward present success risk sev...,2022-09-01,CNN,Julia Robinson,Business,fake
3,You unit its should.,phone which item yard Republican safe where po...,2023-02-07,Reuters,Mr. David Foster DDS,Science,fake
4,Billion believe employee summer how.,wonder myself fact difficult course forget exa...,2023-04-03,CNN,Austin Walker,Technology,fake


In [43]:
# Remove all unneccasary Columns

df = df.drop(columns=['title','date','source','author','category'],axis=1)

In [44]:
type(df)

pandas.core.frame.DataFrame

In [45]:
df

,text,label
0,more tax development both store agreement lawy...,real
1,probably guess western behind likely next inve...,fake
2,them identify forward present success risk sev...,fake
3,phone which item yard Republican safe where po...,fake
4,wonder myself fact difficult course forget exa...,fake
...,...,...
19995,hit and television I change very our happy doo...,fake
19996,fear most meet rock even sea value design stan...,real
19997,activity loss very provide eye west create wha...,real
19998,term point general common training watch respo...,fake


In [46]:
df['label'].value_counts()

,count
label,
fake,10056
real,9944


In [47]:
df.isna().sum()

,0
text,0
label,0


In [71]:
X = df['text']
y = df['label']

le = LabelEncoder()
y = le.fit_transform(y)

vectorizer = TfidfVectorizer(stop_words='english',max_df=0.7)

X_vec = vectorizer.fit_transform(X)

X_train,X_test,y_train,y_test = train_test_split(X_vec,y,test_size=0.2,random_state=42)

In [49]:
models = {
    "Naive Bayes" :  MultinomialNB(),
    "Linear SVM": LinearSVC(),
    "XG Boost" : XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

In [50]:
for name,model in models.items():
  model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  print(f"----{name}----")
  print("Accuracy:", accuracy_score(y_test, y_pred))
  print(classification_report(y_test, y_pred, target_names=le.classes_))


----Naive Bayes----
Accuracy: 0.5085
              precision    recall  f1-score   support

        fake       0.51      0.60      0.55      2029
        real       0.50      0.42      0.46      1971

    accuracy                           0.51      4000
   macro avg       0.51      0.51      0.50      4000
weighted avg       0.51      0.51      0.50      4000

----Linear SVM----
Accuracy: 0.511
              precision    recall  f1-score   support

        fake       0.52      0.51      0.51      2029
        real       0.50      0.51      0.51      1971

    accuracy                           0.51      4000
   macro avg       0.51      0.51      0.51      4000
weighted avg       0.51      0.51      0.51      4000



/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [12:06:53] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


----XG Boost----
Accuracy: 0.51775
              precision    recall  f1-score   support

        fake       0.52      0.52      0.52      2029
        real       0.51      0.52      0.51      1971

    accuracy                           0.52      4000
   macro avg       0.52      0.52      0.52      4000
weighted avg       0.52      0.52      0.52      4000



In [51]:
df.shape

(20000, 2)

In [67]:
vec = TfidfVectorizer(stop_words='english')

In [72]:
def predict_news(text):
    vect = vectorizer.transform([text])
    pred = model.predict(vect)
    return "Fake News" if pred[0] == 1 else "Real News"

# Example usage
print(predict_news("NASA has confirmed aliens have landed in New York NASA has confirmed aliens have landed in New York NASA has confirmed aliens have landed in New York NASA has confirmed aliens have landed in New York"))

Fake News
